# Loading your results 
In the previous tutorials we saw how to load the published data accompanying the paper. Here we'll see how to load results that you create for your own molecules.

## Running your workflow
As explained in the README of this package, you can run your own workflow by navigating to `examples` and running `./run.sh`. To use your own molecules, simply set `smiles_list` to your SMILES strings of interest in `examples/job_info.json`. Other parameters can be changed too; we'll talk about those in the next tutorial.

## Loading workflow results
The workflow results are saved in `examples/summary.pickle`. Let's start with imports and then load the results:

In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
import pickle
import nglview as nv
from ase import Atoms
import numpy as np


In [26]:
path = "../examples/summary.pickle"
with open(path, 'rb') as f:
    info = pickle.load(f)

The data follows the same structure that we saw in previous tutorials:

In [27]:
keys = list(info.keys())
display(keys) # keys are SMILES strings
key = keys[0]

sub_dic = info[key]
display(list(sub_dic.keys()))

['O=S(n1ccc(-c2cc(Cl)cc(-c3ccn(S(=O)C(F)(F)F)c3)c2/N=N\\c2ccccc2)c1)C(F)(F)F',
 'CCN(c1ccccc1)c1cc(C(=O)NOC)ccc1/N=N\\c1ccc(C(=O)NOC)cc1N(CC)c1ccccc1']

['transition_states',
 'trans',
 'cis',
 'results_by_mechanism',
 'summary',
 'cis_smiles',
 'trans_smiles',
 'stable',
 'unstable']

The dictionary contains all the information that was in the `json` files we saw, plus `nxyz` for the geometries. For example:

In [35]:
# get the TS with the lowest free energy

ts_list = []
for this_ts_list in sub_dic['transition_states']:
    ts_list += this_ts_list
    
ts = sorted(ts_list, key=lambda x: x['free_energy'])[0]
display(list(ts.keys()))

['mechanism',
 'confnum',
 'converged',
 'entropy',
 'free_energy',
 'enthalpy',
 'vibfreqs',
 'energy',
 'nxyz',
 'ts_specific_conf_free_energy',
 'ts_specific_avg_conf_energy',
 'ts_specific_conf_entropy',
 'ts_specific_entropy',
 'ts_specific_free_energy',
 'ts_specific_enthalpy',
 'conf_free_energy',
 'avg_conf_energy',
 'conf_entropy',
 'vib_entropy']

We can convert the `nxyz` into an `Atoms` object if we'd like:

In [43]:
nxyz = np.array(ts['nxyz'])
atoms = Atoms(positions=nxyz[:, 1:], 
              numbers=nxyz[:, 0])
display(nv.show_ase(atoms))

NGLWidget()

We see that the lowest-$G$ transition state follows an inversion mechanism. We can check that it's a true TS: 

In [45]:
display(ts['vibfreqs'][:5])

[-405.5635369663351,
 16.461657920271133,
 18.362367198657992,
 20.13337013167,
 27.029220057528676]

And we can compare the TS-specific conformational free energy with the total value:

In [46]:
ha_to_kcal = 627.5
print(ts['ts_specific_conf_free_energy'] * ha_to_kcal)
print(ts['conf_free_energy'] * ha_to_kcal) # only slightly larger in magnitude

-1.7975179372682064
-1.8276939693760301


We can look at the summary of TS quantities:

In [51]:
display({key: val for key, val in sub_dic['summary']['cis'].items() if key not in ['nxyz', 'vibfreqs']})

{'singlet_barrier': {'ts_nxyz': [[8.0,
    1.7841784954071045,
    1.3029667139053345,
    -5.4082350730896],
   [16.0, 3.128742218017578, 1.1112757921218872, -4.8369364738464355],
   [7.0, 2.964113712310791, 0.037922609597444534, -3.5101914405822754],
   [6.0, 3.9357738494873047, -0.8334894776344299, -3.0680935382843018],
   [6.0, 3.3819680213928223, -1.5876020193099976, -2.0851972103118896],
   [6.0, 2.018705129623413, -1.1732686758041382, -1.9191101789474487],
   [6.0, 1.0758988857269287, -1.7275387048721313, -0.9496694803237915],
   [6.0, 1.2137459516525269, -3.0411243438720703, -0.5257799029350281],
   [6.0, 0.3357486128807068, -3.5801122188568115, 0.39435550570487976],
   [17.0, 0.5104588866233826, -5.255257606506348, 0.8619604110717773],
   [6.0, -0.6841373443603516, -2.827714443206787, 0.9396427273750305],
   [6.0, -0.8234296441078186, -1.4905140399932861, 0.5989728569984436],
   [6.0, -1.8535736799240112, -0.6758629679679871, 1.2341456413269043],
   [6.0, -2.2218897342681885, 